In [1]:
import pandas as pd
from flags import FLAGS, unparsed
import numpy as np
from db.conn_db import db,cursor,engine,truncate_table,data_from_mysql,dev_id_train
import operator  
from functools import reduce
import time
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import csr_matrix, hstack
import xgboost as xgb
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss
from data_preprocessing import *

/home/zhijiehuang/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
########## XGBOOST ##########

params = {}
params['booster'] = 'gblinear'
params['objective'] = "multi:softprob"
params['eval_metric'] = 'mlogloss'
params['eta'] = 0.1
params['num_class'] = 22
params['lambda'] = 3
params['alpha'] = 2

train_save = gdbt_data_get_train('n_class')
y_train = train_save['n_class']
targetencoder = LabelEncoder().fit(train_save.n_class)
y = targetencoder.transform(train_save.n_class)
train_save.drop('n_class',axis=1,inplace=True)
X_train = train_save

kf = list(StratifiedKFold(y, n_folds=10, shuffle=True, random_state=4242))[0]

X_train_part, X_val, y_train_part, y_val = train_test_split(X_train, y_train, train_size = 0.8,random_state = 4242)

d_train = xgb.DMatrix(X_train_part, label=y_train_part)
d_valid = xgb.DMatrix(X_val, label=y_val)

watchlist = [(d_train, 'train'), (d_valid, 'eval')]

2018-10-06 09:06:55,556 - DEBUG - data_preprocessing.py:155 - Index(['age', 'device_id', 'sex', 'n_class', 'type_no_w', '01', '02', '02f',
       '1100', '1105',
       ...
       '零售o2o', '音乐学习', '音乐播放器', '音乐编辑', '音乐节奏', '项目管理', '飞行射击', '食谱', '餐饮品牌',
       '驾驶学习'],
      dtype='object', length=875)
2018-10-06 09:06:55,558 - DEBUG - data_preprocessing.py:156 - (50000, 875)
2018-10-06 09:06:55,997 - DEBUG - data_preprocessing.py:178 - Index(['n_class', 'type_no_w', '01', '02', '02f', '1100', '1105', '1107', '1s',
       '2013022',
       ...
       '零售o2o', '音乐学习', '音乐播放器', '音乐编辑', '音乐节奏', '项目管理', '飞行射击', '食谱', '餐饮品牌',
       '驾驶学习'],
      dtype='object', length=872)
2018-10-06 09:06:55,998 - DEBUG - data_preprocessing.py:179 - (50000, 872)
2018-10-06 09:06:55,999 - DEBUG - data_preprocessing.py:180 -    n_class  type_no_w   01   02  02f  1100  1105  1107   1s  2013022  ...   \
0        3   0.599425  0.0  0.0  0.0   0.0   0.0   0.0  0.0      0.0  ...    
1        5   0.667416  0.0  0.

In [3]:
clf = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=25)
X_test = gdbt_data_get_test()
dtrain_predprob = clf.predict(xgb.DMatrix(X_test))

columns=[]
for i in [1,2]:
    for j in range(11):
        columns.append(str(i)+'-'+str(j))
y_pred=pd.DataFrame(dtrain_predprob,columns=columns)
def c(line):
    return [round(x,6) for x in line]
y_pred.apply(lambda line:c(line),axis=1)


#            y_pred=np.array(y_pred).reshape(-1,1)
logging.debug(y_pred)
test_id=pd.read_csv(FLAGS.file_path+'deviceid_test.csv')
logging.debug(test_id['device_id'].shape)
test_id['device_id']=test_id['device_id'].map(str)
test_id.rename(columns={'device_id':'DeviceID'}, inplace = True)
fin=pd.concat([test_id,y_pred],axis=1)
fin.to_csv(FLAGS.tmp_data_path+'1-gblinear.test.csv',index=False)

[0]	train-mlogloss:3.05112	eval-mlogloss:3.05186
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 25 rounds.
[1]	train-mlogloss:3.02131	eval-mlogloss:3.02269
[2]	train-mlogloss:2.9985	eval-mlogloss:3.00045
[3]	train-mlogloss:2.98044	eval-mlogloss:2.9829
[4]	train-mlogloss:2.96607	eval-mlogloss:2.96899
[5]	train-mlogloss:2.95444	eval-mlogloss:2.95776
[6]	train-mlogloss:2.94495	eval-mlogloss:2.94862
[7]	train-mlogloss:2.93715	eval-mlogloss:2.94115
[8]	train-mlogloss:2.93072	eval-mlogloss:2.935
[9]	train-mlogloss:2.92538	eval-mlogloss:2.92993
[10]	train-mlogloss:2.92093	eval-mlogloss:2.92573
[11]	train-mlogloss:2.91721	eval-mlogloss:2.92222
[12]	train-mlogloss:2.91408	eval-mlogloss:2.9193
[13]	train-mlogloss:2.91145	eval-mlogloss:2.91685
[14]	train-mlogloss:2.90923	eval-mlogloss:2.91479
[15]	train-mlogloss:2.90734	eval-mlogloss:2.91305
[16]	train-mlogloss:2.90573	eval-mlogloss:2.91158
[17]	train-mlo

[161]	train-mlogloss:2.89067	eval-mlogloss:2.89848
[162]	train-mlogloss:2.89066	eval-mlogloss:2.89847
[163]	train-mlogloss:2.89065	eval-mlogloss:2.89846
[164]	train-mlogloss:2.89064	eval-mlogloss:2.89846
[165]	train-mlogloss:2.89063	eval-mlogloss:2.89845
[166]	train-mlogloss:2.89062	eval-mlogloss:2.89845
[167]	train-mlogloss:2.89061	eval-mlogloss:2.89844
[168]	train-mlogloss:2.8906	eval-mlogloss:2.89843
[169]	train-mlogloss:2.89059	eval-mlogloss:2.89843
[170]	train-mlogloss:2.89058	eval-mlogloss:2.89842
[171]	train-mlogloss:2.89057	eval-mlogloss:2.89841
[172]	train-mlogloss:2.89055	eval-mlogloss:2.89841
[173]	train-mlogloss:2.89054	eval-mlogloss:2.8984
[174]	train-mlogloss:2.89053	eval-mlogloss:2.89839
[175]	train-mlogloss:2.89052	eval-mlogloss:2.89839
[176]	train-mlogloss:2.89052	eval-mlogloss:2.89838
[177]	train-mlogloss:2.89051	eval-mlogloss:2.89837
[178]	train-mlogloss:2.8905	eval-mlogloss:2.89837
[179]	train-mlogloss:2.89049	eval-mlogloss:2.89836
[180]	train-mlogloss:2.89048	eval-

2018-10-06 09:13:42,855 - DEBUG - data_preprocessing.py:217 - Index(['device_id', 'type_no_w', '01', '02', '02f', '1100', '1105', '1107',
       '1s', '2013022',
       ...
       '零售o2o', '音乐学习', '音乐播放器', '音乐编辑', '音乐节奏', '项目管理', '飞行射击', '食谱', '餐饮品牌',
       '驾驶学习'],
      dtype='object', length=872)
2018-10-06 09:13:42,856 - DEBUG - data_preprocessing.py:218 - (22727, 872)
2018-10-06 09:13:42,900 - DEBUG - data_preprocessing.py:231 - Index(['type_no_w', '01', '02', '02f', '1100', '1105', '1107', '1s', '2013022',
       '2013023',
       ...
       '零售o2o', '音乐学习', '音乐播放器', '音乐编辑', '音乐节奏', '项目管理', '飞行射击', '食谱', '餐饮品牌',
       '驾驶学习'],
      dtype='object', length=871)
2018-10-06 09:13:42,902 - DEBUG - data_preprocessing.py:232 - (22727, 871)
2018-10-06 09:13:42,903 - DEBUG - data_preprocessing.py:233 -    type_no_w   01   02  02f  1100  1105  1107   1s  2013022  2013023  ...   \
0   0.613990  0.0  0.0  0.0   0.0   0.0   0.0  0.0      0.0      0.0  ...    
1   0.639269  0.0  0.0  0.0   

2018-10-06 09:13:44,625 - DEBUG - <ipython-input-3-7252c1eb9d58>:18 - (22727,)
